In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor as rfr



In [ ]:
# Filter the data

In [ ]:
# Dummy data
X = np.random.rand(30,5)
y = np.random.rand(30,1)


In [ ]:
# Split the data (randomly) into train, val and test
X_train, X_test, y_train, y_test 
    = train_test_split(X, y, test_size=0.2, random_state=1)

 X_train, X_val, y_train, y_val 
    = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2
    

In [ ]:
# Scale the data
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)

In [ ]:
# Fitting using chosen method